In [1]:
import os
import pandas as pd
from datasets import Dataset, load_from_disk

In [ ]:
# prefix = "./data/Empathy Data"

# empty_df = pd.DataFrame(columns=['id', 'utterance_id', 'utterance_type', 'utterance_text', 'terminate', 'regDate', 'updDate', 'emotion', 'relation', 'conv_num'])

# tsv_Path_list = []

# for folder_name in os.listdir(prefix):
#     for tsv_file in os.listdir(os.path.join(prefix, folder_name)):
#         tsv_path = os.path.join(prefix, folder_name, tsv_file)
#         tsv_Path_list.append(tsv_path) # 디버깅용
#         df = pd.read_csv(tsv_path, sep='\t')
#         split_name = tsv_file.split('.')[0].split('_')
#         df['emotion'] = split_name[1]
#         df['relation'] = split_name[2]
#         df['conv_num'] = split_name[-1]
#         empty_df = pd.concat([empty_df, df])

In [ ]:
# empty_df.to_csv('empathyDF.csv', index=False)

In [4]:
empathyDF = pd.read_csv('./data/empathyDF.csv')
empathyDF

,id,utterance_id,utterance_type,utterance_text,terminate,regDate,updDate,emotion,relation,conv_num
0,BE22019815,1,0,"동생아, 친구가 요번에 부자 며느리를 맞았다고 자랑이 넘친다. 며느리 덕분에 어깨에...",N,2022-09-22 07:18:56,2022-09-22 07:18:56,상처,형제,288
1,BE22019815,2,1,"언니, 사촌이 땅을 사면 배가 아픈 것처럼 원래 가까운 사이일수록 신경이 더 쓰이는...",N,2022-09-22 07:18:56,2022-11-08 05:27:10,상처,형제,288
2,BE22019815,3,0,난 통화하고 싶지 않은데 자기가 전화해서 계속 자랑이다. 듣다 보면 나중에는 머리가...,N,2022-09-22 07:18:56,2022-09-22 07:18:56,상처,형제,288
3,BE22019815,4,1,언니 친구가 그동안 어렵게 살다 부자 며느리를 얻으니 자랑이 하고 싶은 모양이야. ...,N,2022-09-22 07:18:56,2022-11-08 05:27:10,상처,형제,288
4,BE22019815,5,0,우리 며느리야 잘 얻었다고 생각하지만 경제적인 걸로 자꾸 비교하니 속상한 거야. 어...,N,2022-09-22 07:18:56,2022-11-08 05:27:10,상처,형제,288
...,...,...,...,...,...,...,...,...,...,...
378558,BE22007074,11,0,"네, 이제 석 달이 되었는데 회사에 가는 발걸음이 아주 가볍습니다.",N,2022-07-08 04:39:58,2022-11-08 04:10:03,기쁨,지인,178
378559,BE22007074,12,1,저도 일이 만족스러우면 만사가 즐거운 기분이 듭니다. 그러니 그 발걸음이 깃털만큼 ...,N,2022-07-08 04:39:58,2022-11-08 04:10:03,기쁨,지인,178
378560,BE22007074,13,0,맞습니다. 마음이 가볍고 제가 내린 결정에 대한 만족스러움 덕분인지 기쁜 생각이 줄...,N,2022-07-08 04:39:58,2022-11-08 04:10:03,기쁨,지인,178
378561,BE22007074,14,1,참 좋네요. 적성에 맞는 일을 찾으신 것 같아서 말입니다. 어려운 일이 생겨도 그런...,N,2022-07-08 04:39:58,2022-11-08 04:10:03,기쁨,지인,178


In [5]:
def map_role(type_id):
    # type 0은 사용자, type 1은 로봇의 답변으로 지정
    return "assistant" if type_id == 1 else "user"

def group_to_messages(group):
    messages = []

    group = group.sort_values(by='utterance_id')

    for _, row in group.iterrows():
        messages.append({
            "role": map_role(row['utterance_type']),
            "content": row['utterance_text']
        })
    return {"messages": messages}

In [6]:
df_grouped = empathyDF.groupby('id').apply(group_to_messages).reset_index(name='conversation')

/tmp/ipykernel_4991/506488709.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_grouped = empathyDF.groupby('id').apply(group_to_messages).reset_index(name='conversation')


In [7]:
dataset_list = df_grouped['conversation'].tolist()

In [8]:
dataset = Dataset.from_list(dataset_list)

In [9]:
dataset.save_to_disk("empathyDS")

Saving the dataset (0/1 shards):   0%|          | 0/25456 [00:00<?, ? examples/s]

In [98]:
ds = load_from_disk("./empathyDS")
ds

Dataset({
    features: ['messages'],
    num_rows: 25456
})